# Pandas и присоединение таблиц

In [191]:
import pandas as pd

In [93]:
# Прописываем пути к файлам
bronzefpth = os.path.join('data','medals','Bronze.csv')
silverfpth = os.path.join('data','medals','Silver.csv')
goldfpth = os.path.join('data','medals','Gold.csv')
# Подгружаем CSV
bronze = pd.read_csv(bronzefpth).head(5)
silver = pd.read_csv(silverfpth).head(5)
gold = pd.read_csv(goldfpth).head(5)
gold

,NOC,Country,Total
0,USA,United States,2088.0
1,URS,Soviet Union,838.0
2,GBR,United Kingdom,498.0
3,FRA,France,378.0
4,GER,Germany,407.0


In [94]:
silver

,NOC,Country,Total
0,USA,United States,1195.0
1,URS,Soviet Union,627.0
2,GBR,United Kingdom,591.0
3,FRA,France,461.0
4,GER,Germany,350.0


In [95]:
bronze

,NOC,Country,Total
0,USA,United States,1052.0
1,URS,Soviet Union,584.0
2,GBR,United Kingdom,505.0
3,FRA,France,475.0
4,GER,Germany,454.0


In [96]:
medals = gold.copy()
new_labels = ['NOC', 'Country', 'Gold']
medals.columns = new_labels
medals['Silver'] = silver['Total']
medals['Bronze'] = bronze['Total']

In [97]:
medals.head()

,NOC,Country,Gold,Silver,Bronze
0,USA,United States,2088.0,1195.0,1052.0
1,URS,Soviet Union,838.0,627.0,584.0
2,GBR,United Kingdom,498.0,591.0,505.0
3,FRA,France,378.0,461.0,475.0
4,GER,Germany,407.0,350.0,454.0


In [98]:
# Когда мы хотим слить несколько табиц с одинаковыми колонками в одну большую (по вертикали)
medals_dict = {'gold': goldfpth, 'silver': silverfpth,'bronze':bronzefpth}

In [99]:
# Можно использовать метод df.append()
# !!! Важно, что он работает не так как append в списках, то есть не inplace
medals_df = pd.DataFrame()
for medal, fpth in medals_dict.items():
    df_temp = pd.read_csv(fpth)
    df_temp['medal'] = medal
    medals_df = medals_df.append(df_temp)

# Практическая часть

In [212]:
# Даны 3 таблицы
medalists = pd.read_csv('data/Summer Olympic medalists 1896 to 2008 - ALL MEDALISTS.tsv',
                        sep='\t', skiprows=4)
editions = pd.read_csv('data/Summer Olympic medalists 1896 to 2008 - EDITIONS.tsv', sep='\t')
countries = pd.read_csv('data/Summer Olympic medalists 1896 to 2008 - IOC COUNTRY CODES.csv')

In [221]:
# countries.head(3)
# editions.head(3)
# medalists.head(3)

In [ ]:
# Определите гендерное распределение (в процентах) медалистов по странам по годам 
# Выведите результаты в виде таблицы вместе с полем ISO code
#----- Ваш код здесь ----

# Немного про Pandas & Excel

<img src='img\batman_xl.jpg' width=400 align=left>


In [100]:
# Сохраним medals_df в файл excel
medals_df.to_excel('medals.xlsx', index=False)

In [142]:
# Если в excel - файле только один лист, или нужен только первый
df = pd.read_excel('medals.xlsx')
df.head()

,NOC,Country,Total,medal
0,USA,United States,2088.0,gold
1,URS,Soviet Union,838.0,gold
2,GBR,United Kingdom,498.0,gold
3,FRA,France,378.0,gold
4,GER,Germany,407.0,gold


In [146]:
# Если нужно считать несколько листов из файла, то 
# можно использовать pd.read_excel c параметром sheet_name 
xls = pd.ExcelFile('medals.xlsx')
xls.sheet_names

['Лист1', 'Лист3', 'Лист2']

In [148]:
medals_from_excel = pd.DataFrame()
for sheet_name in xls.sheet_names:
    temp_df = pd.read_excel('medals.xlsx', sheet_name=sheet_name)
    medals_from_excel= medals_from_excel.append(temp_df)

# medals_from_excel

### Как сохранить DataFrame в ексель на разных листах?


In [149]:
# Создаем файл
writer = pd.ExcelWriter('medal_on_sheets.xlsx', engine='xlsxwriter')
# Записываем на листы
gold.to_excel(writer, sheet_name='gold', index=False)
silver.to_excel(writer, sheet_name='silver', index=False)
bronze.to_excel(writer, sheet_name='bronze', index=False)
# Сохраняем и закрвыаем 
writer.save()

In [139]:
# Или так c помощью менеджера контекста with
with pd.ExcelWriter('medal_on_sheets.xlsx', engine='xlsxwriter') as writer:
    for i, tab in enumerate([gold,silver,bronze]):
        tab.to_excel(writer, sheet_name=str(i), index=False)


# Практическая часть 

In [ ]:
# Сохраните DataFrame medalists в файл medalists.xlsx
# В этом файле данные по каждой строне должны быть на отдельном листе
#----- Ваш код здесь ----

# Pandas & ...

<img src="img\web_scrap.jpeg" width=500 align=left>

In [1]:
import pandas as pd
import requests
from tqdm.auto import tqdm

In [2]:
headers = {'X-Requested-With': 'XMLHttpRequest'}
payload = ({'region':'77'})
url = 'http://www.rosneft-azs.ru/map_search'
r = requests.post(url, data=payload, headers=headers)
res = r.json()

In [3]:
stations = res['stations']

In [8]:
stations

[{'id': 56505,
  'name': 'Новосибирскнефтепродукт',
  'brand': 0,
  'rename_fora': 0,
  'floating': 0,
  'address': 'Новосибирская обл., г. Новосибирск, ул. Фабричная, 4а, справа',
  'region_code': 54,
  'contacts': '',
  'email': '',
  'lat': 55.015812,
  'lng': 82.92216,
  'services': {'shop': 0,
   'cafe': 0,
   'wash': 0,
   'tire': 0,
   'hotel': 0,
   'yagas': 1,
   'visa': 0},
  'price': [{'available': 1, 'type': 92, 'price': 43.3, 'fora': 0},
   {'available': 0, 'type': '92_fora', 'price': 0, 'fora': 0},
   {'available': 1, 'type': 95, 'price': 46.25, 'fora': 0},
   {'available': 0, 'type': '95_fora', 'price': 0, 'fora': 0},
   {'available': 0, 'type': '95_jet', 'price': 0, 'fora': 0},
   {'available': 0, 'type': 98, 'price': 0, 'fora': 0},
   {'available': 0, 'type': '98_fora', 'price': 0, 'fora': 0},
   {'available': 0, 'type': 100, 'price': 0, 'fora': 0},
   {'available': 0, 'type': '100_pulsar', 'price': 0, 'fora': 0},
   {'available': 1, 'type': 'diesel', 'price': 51.5, 'f

In [7]:
# Формируем словарь со списками для колонок таблицы
adict = {'Azs':[],'Address' :[],'Price92':[],'Price95':[]}
# Заполняем списки с помощью цикла
for station in tqdm(stations):     
    adict['Azs'].append(station['id'])
    adict['Address'].append(station['address'])
    for d in station['price']:
        if d['type'] == 92:
            adict['Price92'].append(d['price'])
        elif d['type'] == 95:
            adict['Price95'].append(d['price'])

In [6]:
rn = pd.DataFrame.from_dict(adict)
rn.head()

,Azs,Address,Price92,Price95
0,56505,"Новосибирская обл., г. Новосибирск, ул. Фабрич...",43.30,46.25
1,58520,"Ростовская обл., Мясниковский р-н, п. Чалтырь,...",45.44,0.00
2,58316,"Владимирская обл., Судогодский р-н, д. Лаврово...",43.60,46.90
3,58519,"Ростовская обл., г. Ростов-на-Дону, ул. 40 Лет...",45.44,49.74
4,58509,"Калужская обл., г. Калуга, ул. Зерновая, 34",42.80,46.00


# Практическая часть..

In [ ]:
# Добавить в таблицу другие виды топлива
#----- Ваш код здесь ----

In [ ]:
# Отделить регионы от адреса АЗС
#----- Ваш код здесь ----

In [ ]:
# Выбрать АЗС из Белоруссии и сохранить в rnbelarus
#----- Ваш код здесь ----

In [ ]:
# Пересчитать цены rnbelarus в рубли
#----- Ваш код здесь ----

In [ ]:
# Убрать АЗС из Белоруссии из таблциы rn
#----- Ваш код здесь ----

In [ ]:
# Расчитать средние цены на топливо по регионам
#----- Ваш код здесь ----

In [ ]:
# Создать директорию regional_prices
#----- Ваш код здесь ----

In [ ]:
# Записать в отдельный .csv файл цены для каждого региона. 
# Разделить должен быть знак табуляции sep='\t'
# Название файла записать как название региона, заменив пробел на _
#----- Ваш код здесь ----

In [ ]:
# Выведите содержимое директории в переменную list_of_files
#----- Ваш код здесь ----

In [ ]:
# Создайте словарь regional_prices и заполните его таким образом, чтобы
# ключ был название файла без .csv, а значения - список строк, счиатанных из файла
#----- Ваш код здесь ----

In [ ]:
# Соберите таблицу rnnew из словаря regional_prices
#----- Ваш код здесь ----